## Строим вопрос-ответного бота по технологии Retrieval-Augmented Generation на LangChain

[LangChain](https://python.langchain.com) - это набирающая популярность библиотека для работы с большими языковыми моделями и для построения конвейеров обработки текстовых данных. В одной библиотеке присутствуют все элементы, которые помогут нам создать вопрос-ответного бота на наборе текстовых данных: вычисление эмбеддингов, запуск больших языковых моделей для генерации текста, поиск по ключу в векторных базах данных и т.д.

Для начала, установим `langchain` и сопутствующие библиотеки.

In [ ]:
# Ok #################################

%pip install -q langchain sentence_transformers lancedb unstructured

## Загружаем документы и разбиваем на части

Для загрузки документов используется `DirectoryLoader`, который загружает файлы из указанной диреткории. Поддерживаются загрузка различных типов файлов - документацию по загрузчику смотри [здесь](https://langchain-fanyi.readthedocs.io/en/latest/modules/indexes/document_loaders/examples/directory_loader.html). 

Можно использовать загрузчик *файлов* `UnstructuredFileLoader` (по-умолчанию, `DirectoryLoader` использует этот метод), в параметрах которого можно задать точноcть разбиения документов. Загрузчик неструктурированных документов позволяет пользователям передавать strategy параметр, который позволяет unstructured узнать, как разбить документ на разделы. В настоящее время поддерживаются следующие стратегии: "hi_res" (по умолчанию) и "fast". Стратегии разбиения на разделы в высоком разрешении более точны, но требуют больше времени для обработки. Быстрые стратегии позволяют быстрее разбивать документ на разделы, но при этом снижают точность. Не для всех типов документов существуют отдельные стратегии разбиения в высоком разрешении и быстром режиме. Для этих типов документов strategy параметр kwarg игнорируется. В некоторых случаях стратегия высокого разрешения будет заменена на fast, если отсутствует зависимость (например, модель разделения документа).

**Unstructured** создает разные “элементы” для разных фрагментов текста. По умолчанию мы объединяем их вместе, но вы можете легко сохранить это разделение, указав mode="elements".

В настоящее время **Unstructured** поддерживает загрузку текстовых файлов, Powerpoint, HTML, PDF-файлов, изображений и многого другого.

Для работы **retrival augmented generation** нам необходимо по запросу найти наиболее релевантные фрагменты исходного текста, на основе которых будет формироваться ответ. Для этого нам надо разбить текст на такие фрагменты, по которым мы сможем вычислять эмбеддинг, и которые будут с запасом помещаться во входное окно используемой большой языковой модели.

Для этого можно использовать механизмы фреймворка **LangChain** - например, `RecursiveCharacterTextSplitter`. Он разбивает текст на перекрывающиеся фрагменты по набору типовых разделителей - абзацы, переводы строк, разделители слов.

> **ВАЖНО**: Перед выполнением ячейки не забудьте установить имя пользователя, которое вы использовали на предыдущем шаге.

Размер `chunk_size` нужно выбирать исходя из нескольких показателей:
* Допустимая длина контекста для эмбеддинг-модели. Yandex GPT Embeddings допускают 2048 токенов, в то время как многие открытые модели HuggingFace имеют длину контекста 512-1024 токена
* Допустимый размер окна контекста большой языковой модели. Если мы хотим использовать в запросе top 3 результатов поиска, то 3 * chunk_size+prompt_size+response_size должно не превышать длины контекста модели.

In [ ]:
# Ok #################################

import langchain
import langchain.document_loaders
# про сплиттер
# https://python.langchain.com/docs/integrations/vectorstores/vald#basic-example
from langchain.text_splitter import CharacterTextSplitter

In [ ]:
# Ok #################################

#user = 'shwars'
chunk_size = 1024   # 2048
chunk_overlap= 128  #50
#source_dir = f"/home/jupyter/datasphere/s3/s3store/{user}/text"
source_dir = 'txts'

loader = langchain.document_loaders.DirectoryLoader(source_dir,glob="*.txt",show_progress=True,recursive=True)
splitter = langchain.text_splitter.RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)
fragments = splitter.create_documents([ x.page_content for x in loader.load()])
print('Кол-во фрагментов текстов:',len(fragments))

## Вычисляем эмбеддинги для всех фрагментов

Для вычисления эмбеддингов можно взять какую-нибудь модель из репозитория `HuggingFace`, с поддержкой русского языка. `LangChain` содержит свои встроенные средства работы с эмбеддингами, и поддерживает модели из `HuggingFace`: 

In [ ]:
# ЭТО ПРИМЕР с сайта 
# https://python.langchain.com/docs/integrations/vectorstores/vald#basic-example

from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Vald

raw_documents = TextLoader("state_of_the_union.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)
embeddings = HuggingFaceEmbeddings()
db = Vald.from_documents(documents, embeddings, host="localhost", port=8080)

In [ ]:
%pip install -U sentence-transformers


In [ ]:
# Ok #################################
# Этотак просто, для проверки возможностей эмбединга 

embeddings = langchain.embeddings.HuggingFaceEmbeddings(model_name="distiluse-base-multilingual-cased-v1")
sample_vec = embeddings.embed_query("Hello, world!")
len(sample_vec)

Можно использовать более продвинутую модель [эмбеддингов от Yandex GPT](https://cloud.yandex.ru/docs/yandexgpt/api-ref/Embeddings/). Вот так можно вызывать её через API Yandex Cloud. Не забудьте при необходимости исправить параметры `api_key` и `folder_id` в соответствии с вашими данными.

In [ ]:
# Ok #################################
# Лучше использовать json-конфиг
import os
#api_key = os.environ['api_key']
api_key = ""
folder_id = ""

In [ ]:
# НЕ ИСПОЛЬЗУЕМ __________________________________________________________________________
import requests

headers={ 
    "Authorization" : f"Api-key {api_key}",
    "x-folder-id" : folder_id
}
j = {
  "model" : "general:embedding",
  "embedding_type" : "EMBEDDING_TYPE_DOCUMENT",
  "text": "Hello, world!"
}

res = requests.post("https://llm.api.cloud.yandex.net/llm/v1alpha/embedding",json=j,headers=headers)
vec = res.json()['embedding']
print("Длина вектора",len(vec))

In [ ]:
%pip install yandex-chain

In [ ]:
%pip install faiss-cpu

In [ ]:
# ПРИМЕР ИЗ ДОКУМЕНТАЦИИ  ##############################

from yandex_chain import YandexLLM, YandexEmbeddings
#from langchain.vectorstores import FAISS
from langchain_community.vectorstores import FAISS


docs = "How are you today?"
embeddings = YandexEmbeddings(config="config.json")
vectorstore = FAISS.from_texts(docs, embedding=embeddings)
retriever = vectorstore.as_retriever()

## Cохраняем эмбеддинги  в векторную БД

Для поиска нам нужно уметь быстро сопоставлять эмбеддинг запроса, и эмбеддинги всех фрагементов наших исходных материалов. Для этого используются векторные базы данных. Для крупных проектов имеет смысл использовать серьезные инструменты, типа [OpenSearch](https://opensearch.org/) (доступный [в виде сервиса в облаке Yandex Cloud](https://cloud.yandex.ru/services/managed-opensearch)), но для нашего примера мы используем небольшую векторную БД [LanceDB](https://lancedb.com/), хранящую индекс в директории на диске. (Примеры использования различных БД можно посмотреть [здесь](https://python.langchain.com/docs/integrations/vectorstores))

Первоначально создадим базу данных, добавив туда одну строчку:

In [ ]:
# Ok ### ВКЛЮЧАЕМ В КОД
# Первоначально создадим базу данных, добавив туда одну строчку

#from langchain.vectorstores import LanceDB
import lancedb
import os
#import tqdm as notebook_tqdm
from tqdm.notebook import tqdm
from yandex_chain import YandexLLM, YandexEmbeddings


embeddings = YandexEmbeddings(config="config.json")

db_dir = "store"

os.makedirs(db_dir,exist_ok=True)

db = lancedb.connect(db_dir)

table = db.create_table(
    "vector_index",
    data=[
        {
            "vector": embeddings.embed_query("Hello World"),
            "text": "Hello World",
            "id": "1",
        }
    ],
    mode="overwrite",
)

Далее проиндексируем все выделенные ранее фрагменты нашей документации. 

> В зависимости от объема текста, эта ячейка может выполняться достаточно длительное время - вспомним про задержку в 1 сек между запросами!

In [ ]:
# Ok ### ВКЛЮЧАЕМ С КОД

db = LanceDB.from_documents(fragments, embeddings, connection=table)

In [ ]:
# требуется для выполнения запрос-ответ
%pip install pandas

Теперь посмотрим, насколько хорошо находятся фрагменты текста по какому-то запросу:

In [ ]:
# Ok #################################
# ВКЛЮЧАЕМ В КОД СТОРОЙ СПОСОБ - ячейка ниже

q="Можно ли провозить лыжи на борту самолёта?"
# q="Я инвалид-колясочник. Что мне нужно сделать, чтобы мне помогли в аэропорту?"
#q="Спасибо"


res = db.similarity_search(q)
for x in res:
    print('-'*50)
    print(x.page_content)

In [ ]:
#%pip install str

Ещё один полезный интерфейс для поиска текстов - это `Retriever`, убедимся, что он тоже работает:

In [ ]:
# Ok ВКЛЮЧАЕМ В КОД #################################

#q="Можно ли провозить лыжи на борту самолёта?"
q="Я инвалид-колясочник. Что мне нужно сделать, чтобы мне помогли в аэропорту?"

retriever = db.as_retriever(
    search_kwargs={"k": 20}
)
res = retriever.get_relevant_documents(q)

concatdocs = '\n'
for x in res:
    print(x.page_content)
    print('________')
#    concatdocs += x.page_content

#print(concatdocs)



## Примечание
### Подбор `search_kwargs`
Опыт показал, что требуется подбор `search_kwargs`. Для текущей задачи при k=5 бот не даёт ответ на вопрос __Я инвалид-колясочник. Что мне нужно сделать, чтобы мне помогли в аэропорту?__. 

Ответ получается такой: __К сожалению, я не могу ничего сказать об этом. Давайте сменим тему?__. 

### Хороший ответ получается при k=15...20.
Пример ответа пр  k=15 представлен ниже. Важно отметить, что при увеличении `k` может ворзникнуть ошибка из-за ограниченний по кол-ву токенов, обрабатываемых моделью.

```md
Согласно правилам перевозки авиакомпании «АЗУР эйр», если вы заранее согласовали данную услугу с авиакомпанией, а ваш багаж прошёл специальный досмотр на авиационную безопасность, то вам должны помочь в аэропорту.

Вы можете согласовать перевозку вашего кресла-коляски как сверхнормативного багажа, если это было заранее согласовано с авиакомпанией и оплачено.

Если у вас есть вопросы, не освещённые в данных правилах, вы можете позвонить по телефону *7 (495) 374-55-14* или отправить запрос на электронную почту `call@azurair.ru`.

Также авиакомпания может предоставить вам информацию о правилах перевозки, в том числе о перевозке кресел-колясок и других вспомогательных устройств передвижения, используемых пассажирами из числа инвалидов и других лиц с ограничениями жизнедеятельности.

Пожалуйста, обратите внимание, что информация, содержащаяся в данном ответе, актуальна на момент публикации. Актуальную информацию вы можете найти на официальном сайте авиакомпании «АЗУР эйр».
```


## Подключаемся к Yandex GPT

Фреймворк LangChain поддерживает интеграцию с различными большими языковыми моделями, но Yandex GPT в их число не входит. Поэтому, как и в случае с эмбеддингами, нам надо написать соответствующий адаптер, предоставляющий доступ к Yandex GPT в формате LangChain. Для подробностей вызова Yandex GPT можно обратиться к документации по [YandexGPT API](https://cloud.yandex.ru/docs/yandexgpt/)

In [ ]:
# --- ЭТО не включаем в код ________________________________________
from typing import Any, List, Mapping, Optional
from langchain.callbacks.manager import CallbackManagerForLLMRun
import requests

class YandexLLM(langchain.llms.base.LLM):
    api_key: str = None
    iam_token: str = None
    folder_id: str
    max_tokens : int = 1500
    temperature : float = 1
    instruction_text : str = None

    @property
    def _llm_type(self) -> str:
        return "yagpt"

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
    ) -> str:
        if stop is not None:
            raise ValueError("stop kwargs are not permitted.")
        headers = { "x-folder-id" : self.folder_id }
        if self.iam_token:
            headers["Authorization"] = f"Bearer {self.iam_token}"
        if self.api_key:
            headers["Authorization"] = f"Api-key {self.api_key}"
        req = {
          "model": "general",
          "instruction_text": self.instruction_text,
          "request_text": prompt,
          "generation_options": {
            "max_tokens": self.max_tokens,
            "temperature": self.temperature
          }
        }
        res = requests.post("https://llm.api.cloud.yandex.net/llm/v1alpha/instruct",
          headers=headers, json=req).json()
        return res['result']['alternatives'][0]['text']

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {"max_tokens": self.max_tokens, "temperature" : self.temperature }

In [ ]:
# Ok #################################
# ВКЛЮЧАЕМ В КОД импорт библиотек (скорее всего достаточно импортировтаь from langchain.prompts import ChatPromptTemplate )
# КОД ИЗ ДОКУМЕНТАЦИИ yandex-chain
# НЕ СКЛЮЧАеМ С КОД ________________________________________
from operator import itemgetter

from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

'''
# Отвечай на вопросы только в контексте следующих документв:
#  Answer the question based only on the following context:
template = """ Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = YandexLLM(config="config_neurocat.json", use_lite=False)

chain = (
    {"context": retriever, "question": RunnablePassthrough()} 
    | prompt 
    | model 
    | StrOutputParser()
)
'''

In [ ]:
# код мз документации НЕ СРАБОТАЛ
#query = 'Можно ли провозить аккумуляторы в салоне самолёта?'
chain.invoke(q)

Теперь попросим модель ответить на наш вопрос от лица сотрудника компании:

In [ ]:
# Ok - включаем в код #################################

instructions = """
Ты информационный бот авиакомпании "АЗУР эйр" по имени Роберт. 
Твоя задача - полно и вежливо отвечать на вопросы собеседника.
"""

llm = YandexLLM(config="config.json", use_lite=False, temperature= 0.2,         #  api_key=api_key, folder_id=folder_id,
                instruction_text = instructions)

In [ ]:
# Ok  не нужно включать в код ________________________________________
# Это просто проверка работы модели без привязки к контенту документов
q = 'Можно ли провозить аккумуляторы в салоне самолёта?'
print('Вопрос клиента:', q)
print('Ответ АзурБота:\n',llm(q))

В данном примере мы пока что никак не использовали наши текстовые документы.

## Собираем Retrieval-Augmented Generation

Пришла пора соединить всё вместе и научить бота отвечать на вопросы, подглядывая в наш текстовый корпус. Для этого используем механизм цепочек (*chains*). Основную функциональность реализует `StuffDocumentsChain`, которая делает следующее:

1. Берёт коллекцию документов `input_documents`
1. Каждый из них пропускает через `document_prompt`, и затем объединяет вместе.
1. Данный текст помещается в переменную `document_variable_name`, и передаётся большой языковой модели `llm`

В нашем случае `document_prompt` не будет модицицировать документ, а основной запрос к LLM будет содержать в себе инструкции для Yandex GPT. 

In [ ]:
# Промпт для обработки документов ### ВКЛЮЧАЕМ В КОД
# Ответ на вопрос выдаётся в контексте релевантных документов (fragments), сохранённых в res
from langchain import chains

document_prompt = langchain.prompts.PromptTemplate(
    input_variables=["page_content"], template="{page_content}"
    )

# Промпт для языковой модели
document_variable_name = "context"
'''
stuff_prompt_override = """
Пожалуйста, отвечай на вопрос только в контексте правил воздушных перевозок авиакомпании "АЗУР эйр".
Если ответа в правилах нет, извениьсь и порекомендуй позвонить в контактный ценр по телефону +7(495)374-55-14 или отправить запрос на электронную почту `call@azurair.ru`.
Если собеседник не задал вопрос, поприветствуй его и сообщи о своей готворности ответить на его вопросы.
Вот правила воздушных перевозок:
-----
{context}
-----
Вопрос:
{query}"""
'''
stuff_prompt_override = """
Отвечай на вопрос только в контексте правил воздушных перевозок авиакомпании "АЗУР эйр"
Если в правилах нет ответа на вопрос собеседника, рекомендуй позвонить в контактный ценр по телефону +7(495)374-55-14 или отправить запрос на электронную почту `call@azurair.ru
При ответе учитывай нижеследующие выдержки из правил воздушных перевозок авиакомпании "АЗУР эйр":
-----
{context}
-----
Вопрос:
{query}"""

prompt = langchain.prompts.PromptTemplate(
    template=stuff_prompt_override, input_variables=["context", "query"]
    )

# Создаём цепочку
llm_chain = langchain.chains.LLMChain(llm=llm, prompt=prompt)
chain = langchain.chains.StuffDocumentsChain(
    llm_chain=llm_chain,
    document_prompt=document_prompt,
    document_variable_name=document_variable_name,
    )


from pathlib import Path

# Запуск цепочки
results = chain.run(input_documents=res, query=q)

# Создание пути к файлу
path = Path("answer.md")

# Запись результата в файл
with open(path, "w") as file:
    file.write(results)

# Сообщение о завершении
print(f"Результат записан в файл: {path}")





Чтобы ещё более улучшить результат, мы можем использовать хитрый механизм перемешивания документов, таким образом, чтобы наиболее значимые документы были ближе к началу запроса. Также мы оформим все операции в одну функцию `answer`:

In [ ]:
# OK ==== Сортировка по релевантрости работает
# Разобраться, куда код вставить

from langchain.document_transformers import LongContextReorder
#reorderer = LongContextReorder(temperature=0.05, device='cuda', return_full_response=True)
reorderer = LongContextReorder()

# "Температура" модели. Чем выше значение, тем более "творческими" будут результаты. По умолчанию: 1.0
# return_full_response (bool, optional) Если True, функция будет возвращать полный ответ модели, включая logits, probabilities и hidden states. По умолчанию: False

def answer(query,reorder=True,print_results=False): # если reordr=True, то производится сортировка по степени релевантности
  results = retriever.get_relevant_documents(query)
  if print_results:
        for x in results:
            print(f"{x.page_content}\n--------")
  if reorder:
    results = reorderer.transform_documents(results)
  return chain.run(input_documents=results, query=query)

In [ ]:
result2 = answer(q, reorder=True, print_results=True)

# Создание пути к файлу
path = Path("answer_v2.md")

# Запись результата в файл
with open(path, "w") as file:
    file.write(result2)

# Сообщение о завершении
print(f"Результат записан в файл: {path}")


In [ ]:
# Пример от Gemini ### НЕ ВКЛЮЧАЕМ В КОД
from langchain.document_transformers import LongContextReorder


model = LongContextReorder()

response = model("This is an example sentence.")

print(response)

In [ ]:
# Пример от Gemini ### НЕ ВКЛЮЧАЕМ В КОД
from langchain.document_transformers import LongContextReorder

chain = (
    chain.map(LongContextReorder())
    .map(lambda x: x.text)
)

response = chain("This is an example sentence.")

print(response)

Можно сравнить результаты, выдаваемые Yandex GPT напрямую, с результатами нашей генерации на основе документов:

In [ ]:
# ### НЕ ВКЛЮЧАЕМ В КОД

def compare(q):
    print(f"Ответ YaGPT: {llm(q)}")
    print(f"Ответ бота: {answer(q)}")
    
compare("Что ты можешь сказать правилах перевозки лыж в самолёте?")

## Сохраняем векторную БД в Storage

Для следующего этапа - вопрос-ответного бота - нам потребуется созданная нами база данных с документами. Поэтому скопируем её на хранилище s3:

In [ ]:
!cp -R ./store /home/jupyter/datasphere/s3/s3store/shwars/

# Создаём вопрос-ответного бота в телеграм
Для создания вопрос-ответного бота нам потребуется развернуть нашу цепочку LangChain в виде публично-доступного веб-сервиса по HTTPS. Это удобнее всего сделать с помощью виртуальной машины Yandex Compute. Для понимания того, как устроены боты в телеграм, можно порекомендовать эту документацию.

1. Создаём виртуальную машину. Для экспериментов нам не нужна высокая производительность, будет достаточно 4-6 Gb RAM, 50 Gb SSD, Ubuntu. Для входа на виртуальную машину используется ssh-сертификат.

 > Код телеграм-бота подразумевает, что пользователь на виртуальной машине будем иметь имя vmuser. Если вы используете другое имя, то придётся внести исправления в код.

2. Создаём для виртуальной машины статический IP-адрес
3. Для работы с телеграм потребуется HTTPS-протокол и сертификат SSL. Поэтому необходимо привязать к виртуальной машине какое-то доменное имя.
4. Заходим в консоль виртуальной машины по SSH
5. Клонируем репозиторий проекта `git clone https://github.com/yandex-datasphere/VideoQABot`
6. Переходим в каталог проекта и устанавливаем зависимости:

```bash
cd VideoQABot
pip3 install -r requirements.txt
pip install waitress
```

```bash
Installing collected packages: waitress
  WARNING: The script waitress-serve is installed in '/home/azurbot/.local/bin' which is not on PATH.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
```


7. Создаём SSL-сертификат для выбранного ранее доменного имени, это можно сделать, например, с помощью бесплатного сервиса Let's Encrypt и `certbot`
8. Сертификаты записываем в директорию cert, и прописываем путь к ним в файле `telegram.py`
9. Создаём бота в телеграм при помощи botfather (см. [докeнтацию](https://core.telegram.org/bots/tutorial#getting-ready)), и полученный telegram token записываем в `config.json`
10. Также в `config.json` прописываем адрес нашего сайта. Рекомендуется использовать порт `8443`, поскольку в этом случае запускать веб-сервер можно от имени обычного пользователя.
11. Копируем векторную базу данных, полученную на предыдущем шаге, в директорию store.
12. Запускаем `python3 telegram.py`

На этом этапе вы должны быть в состоянии послать в бота сообщения, текстом или как голосовое сообщение, и получить ответ, текстом + голосом.